In [2]:
#import createInput as create
import sys
import json
import copy
import numpy as np
import matplotlib.pyplot as plt
import pandexo as exo
import exosim_modes as exom
%matplotlib inline
import cProfile
planetfile = '/Users/nbatalh1/Downloads/forNatasha_HD97658b_DLSODES.txt'
pwunits = 'um'
pfunits = 'fp/f*'
starfile = "/Users/nbatalh1/Desktop/JWST/pandexo/gj1132_star.txt"
swunits = 'Angs'
sfunits = 'FLAM'
jmags = [6.9]

for jmag in jmags:
    hd = exom.SetDefaultModes(starfile,planetfile,  jmag)

    spec= hd.nirspec()
    spec['pandexo_input']['star']['type']= 'phoenix'
    spec['pandexo_input']['star']['temp'] = 5750.0
    spec['pandexo_input']['star']['logg'] = 4.51
    spec['pandexo_input']['star']['metal'] = -.23
    spec['pandexo_input']['planet']['w_unit'] = pwunits
    spec['pandexo_input']['planet']['f_unit'] = pfunits
    spec['pandexo_input']['star']['w_unit'] = swunits
    spec['pandexo_input']['star']['f_unit'] = sfunits
    spec['pandexo_input']['observation']['transit_duration'] = .1234*24.0*60.0*60.0

    spec['pandexo_input']['observation']['noccultations'] = 1


    sec3 = spec
    #sec3['pandeia_input']['configuration']['instrument']['filter'] = None
    #sec3['pandeia_input']['configuration']['instrument']['disperser'] = 'gr700xd'
    #sec3['pandeia_input']['calculation']['noise']['rn_correlation'] = True
    section5 = exo.wrapper(spec)
    #print jmag, section5["timing"]['Num Groups per Integration'], np.median(section5['error_unbin'])*1e6

    #a = cProfile.run('exo.wrapper(sec3)', sort='cumtime')

In [2]:
from bokeh.io import vform,output_file, show
import numpy as np
from bokeh.plotting import figure
from bokeh.embed import components
from bokeh.models.widgets import Panel, Tabs
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.charts import HeatMap, color
from bokeh.palettes import brewer
import pandas as pd
import pickle as pkl 
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
ALL = pkl.load(open('/Users/nbatalh1/Desktop/Simulations/NIRISS/WASP-62 b.p', 'rb'))
result_dict = ALL
print ALL.keys()

['error_bin', 'timing_div', 'wave_bin', 'warnings_div', 'sim_spec', 'error_unbin', 'input_div', 'timing', 'wmodel', 'model', 'wave_unbin', 'out']


In [12]:
noccultations = result_dict['timing']['Number of Transits']


#bounds 
#LBout = result_dict['LBout']
#UBout = result_dict['UBout']

#print result_dict['out']['warnings']


In [16]:
output_file("junk.html")

TOOLS = "pan,wheel_zoom,box_zoom,resize,reset,save"

#MAIN spectrum 
punit = result_dict['input']['Primary/Secondary']
p=1.0
if punit == 'fp/f*': p = -1.0
else: punit = '('+punit+')^2'

flux_out = result_dict['RawData']['flux_out']
flux_in = result_dict['RawData']['flux_in']
var_tot = result_dict['RawData']['var_out'] + result_dict['RawData']['var_in']

x = result_dict['FinalSpectrum']['wave']
y = result_dict['FinalSpectrum']['spectrum_w_rand']
err = result_dict['FinalSpectrum']['error_w_floor']

y_err = []
x_err = []
for px, py, yerr in zip(x, y, err):
    np.array(x_err.append((px, px)))
    np.array(y_err.append((py - yerr, py + yerr)))

source = ColumnDataSource(data=dict(x=x, y=y, y_err=y_err, x_err=x_err, err=err, flux_out=flux_out, flux_in=flux_in, var_tot=var_tot, p=var_tot*0+p))
original = ColumnDataSource(data=dict(x=x, y=y, y_err=y_err, x_err=x_err, err=err, flux_out=flux_out, flux_in=flux_in, var_tot=var_tot))

ylims = [min(result_dict['OriginalInput']['og_spec'])- 0.1*min(result_dict['OriginalInput']['og_spec']),
             0.1*max(result_dict['OriginalInput']['og_spec'])+max(result_dict['OriginalInput']['og_spec'])]
xlims = [min(result_dict['FinalSpectrum']['wave']), max(result_dict['FinalSpectrum']['wave'])]

plot_spectrum = figure(plot_width=800, plot_height=300, x_range=xlims,
                           y_range=ylims, tools=TOOLS,#responsive=True,
                             x_axis_label='Wavelength [microns]',
                             y_axis_label=punit, 
                           title="Original Model with Observation")

plot_spectrum.line(result_dict['OriginalInput']['og_wave'],result_dict['OriginalInput']['og_spec'], color= "black", alpha = 0.5, line_width = 4)

plot_spectrum.circle('x', 'y', source=source, line_width=3, line_alpha=0.6)
plot_spectrum.multi_line('x_err', 'y_err', source=source)

callback = CustomJS(args=dict(source=source, original=original), code="""
        // Grab some references to the data
        var sdata = source.get('data');
        var odata = original.get('data');

        // Create copies of the original data, store them as the source data
        sdata['x'] = odata['x'].slice(0);
        sdata['y'] = odata['y'].slice(0);

        sdata['y_err'] = odata['y_err'].slice(0);
        sdata['x_err'] = odata['x_err'].slice(0);
        sdata['err'] = odata['err'].slice(0);

        sdata['flux_out'] = odata['flux_out'].slice(0);
        sdata['flux_in'] = odata['flux_in'].slice(0);
        sdata['var_tot'] = odata['var_tot'].slice(0);

        // Create some variables referencing the source data
        var x = sdata['x'];
        var y = sdata['y'];
        var y_err = sdata['y_err'];
        var x_err = sdata['x_err'];
        var err = sdata['err'];
        var p = sdata['p'];

        var flux_out = sdata['flux_out'];
        var flux_in = sdata['flux_in'];
        var var_tot = sdata['var_tot'];

        var f = cb_obj.get('value');

        var wlength = Math.pow(10.0,f);

        var ind = [];
        ind.push(0);
        var start = 0;


        for (i = 0; i < x.length-1; i++) {
            if (x[i+1] - x[start] >= wlength) {
                ind.push(i+1);
                start = i;
            }
        }

        if (ind[ind.length-1] != x.length) {
            ind.push(x.length);
        }

        var xout = [];


        var foutout = [];
        var finout = [];
        var varout = [];

        var xslice = []; 


        var foutslice = [];
        var finslice = [];
        var varslice = [];

        function add(a, b) {
            return a+b;
        }

        for (i = 0; i < ind.length-1; i++) {
            xslice = x.slice(ind[i],ind[i+1]);

            foutslice = flux_out.slice(ind[i],ind[i+1]);
            finslice = flux_in.slice(ind[i],ind[i+1]);
            varslice = var_tot.slice(ind[i],ind[i+1]);

            xout.push(xslice.reduce(add, 0)/xslice.length);
            foutout.push(foutslice.reduce(add, 0));
            finout.push(finslice.reduce(add, 0));
            varout.push(varslice.reduce(add, 0));

            xslice = [];

            foutslice = [];
            finslice = [];
            varslice = [];
        }

        for (i = 0; i < x.length; i++) {
            y[i] = p[i]*(foutout[i]-finout[i]+ (Math.sqrt(varout[i])*(Math.random()-Math.random())))/foutout[i]; 
            x[i] = xout[i];
            x_err[i][0] = xout[i];
            x_err[i][1] = xout[i];
            y_err[i][0] = y[i] + (Math.sqrt(varout[i])/foutout[i]);
            y_err[i][1] = y[i] -(Math.sqrt(varout[i])/foutout[i]);            
        }

        source.trigger('change');
    """)

slider =  Slider(title="binning", value=np.log10(x[1]-x[0]), start=np.log10(x[1]-x[0]), end=np.log10(max(x)/2.0), step= .05, callback=callback)
layout = vform(slider, plot_spectrum)
show(layout)                                                          

In [2]:
def bokeherrorbar(fig, x, y, xerr=None, yerr=None, color='red', point_kwargs={}, error_kwargs={}):

  fig.circle(x, y, size = 5, color=color, **point_kwargs)

  if xerr is not None:
      x_err_x = []
      x_err_y = []
      for px, py, err in zip(x, y, xerr):
          x_err_x.append((px - err, px + err))
          x_err_y.append((py, py))
      fig.multi_line(x_err_x, x_err_y, color=color, **error_kwargs)

  if yerr is not None:
      y_err_x = []
      y_err_y = []
      for px, py, err in zip(x, y, yerr):
          y_err_x.append((px, px))
          y_err_y.append((py - err, py + err))
      fig.multi_line(y_err_x, y_err_y, color=color, **error_kwargs)

In [24]:
output_file("slider.html")
TOOLS = "pan,wheel_zoom,box_zoom,resize,reset,save"
# Flux 1d
x, y = out['1d']['extracted_flux']
x = x[~np.isnan(y)]
y = y[~np.isnan(y)]


plot_flux_1d1 = figure(tools=TOOLS,#responsive=True,
                     x_axis_label='Wavelength [microns]',
                     y_axis_label='Flux', title="Out of Transit Flux",
                     plot_width=1000, plot_height=300)
plot_flux_1d1.line(x, y, line_width = 4, alpha = .7)
tab1 = Panel(child=plot_flux_1d1, title="Total Flux")

# BG 1d
x, y = out['1d']['bg']
x = x[~np.isnan(y)]
y = y[~np.isnan(y)]

plot_bg_1d1 = figure(tools=TOOLS,#responsive=True,
                     x_axis_label='Wavelength [microns]',
                     y_axis_label='Flux', title="Background",
                     plot_width=1000, plot_height=300)
plot_bg_1d1.line(x, y, line_width = 4, alpha = .7)
tab2 = Panel(child=plot_bg_1d1, title="Background Flux")

# SNR 1d
x, y = out['1d']['sn']
x = x[~np.isnan(y)]
y = y[~np.isnan(y)]


plot_snr_1d1 = figure(tools=TOOLS,#responsive=True,
                     x_axis_label='Wavelength [microns]',
                     y_axis_label='SNR', title="Signal-to-Noise Ratio",
                     plot_width=500, plot_height=250)
plot_snr_1d1.line(x, y, line_width = 4, alpha = .7, color='black')
tab3 = Panel(child=plot_snr_1d1, title="SNR")


# Error bars (ppm) 

x = result_dict['wave_bin']
y = result_dict['error_unbin']*1e6
x = x[~np.isnan(y)]
y = y[~np.isnan(y)]


ymed = np.median(y)
#y[(y>0.01) & (y<5000)] = 0.0
#x[(y>0.01) & (y<5000)]

plot_noise_1d1 = figure(tools=TOOLS,#responsive=True,
                     x_axis_label='Wavelength [microns]',
                     y_axis_label='Error on Spectrum (PPM)', title="Error Curve",
                     plot_width=1000, plot_height=300, y_range = [0,2.0*ymed])

plot_noise_1d1.circle(x, y, line_width = 4, alpha = .7)
tab4 = Panel(child=plot_noise_1d1, title="Error")

#Not happy? Need help picking a different mode? 
plot_spectrum2 = figure(plot_width=1000, plot_height=300, x_range=[0.5,14], tools=TOOLS,
                         x_axis_label='Wavelength [microns]',
                         y_axis_label='Rp/R*', title="Original Model with Observing Efficiency",y_axis_type="log")

plot_spectrum2.line(result_dict['wmodel'],model, line_width = 4,alpha = .7)
tab5 = Panel(child=plot_spectrum2, title="Not Happy? Choose another mode")




tabs1d = Tabs(tabs=[ tab1, tab2,tab3, tab4, tab5])
#from bokeh.embed import components
#script, div = components({'tab5':tab5})

show(tabs1d)

In [ ]:
print result_dict['out']['warnings']

# 2 D PLOTS

In [ ]:
output_file("junk.html")
data = out['2d']['detector']
TOOLS = "pan,wheel_zoom,box_zoom,resize,reset,save"


xr, yr = data.shape

plot_detector_2d = figure(responsive=True, tools=TOOLS,
                     x_range=[0, yr], y_range=[0, xr],
                     x_axis_label='Pixel', y_axis_label='Spatial',
                     title="2D Detector Image",
                    plot_width=1000, plot_height=300)
plot_detector_2d.image(image=[data], x=[0], y=[0], dh=[xr], dw=[yr],
                  palette="Spectral11")


show(plot_detector_2d)

In [25]:
#2d tabs 

#snr 
data = out['2d']['snr']
data[np.isinf(data)] = 0.0
xr, yr = data.shape
plot_snr_2d = figure(tools=TOOLS,
                     x_range=[0, yr], y_range=[0, xr],
                     x_axis_label='Pixel', y_axis_label='Spatial',
                     title="Signal-to-Noise Ratio",
                    plot_width=500, plot_height=250)
plot_snr_2d.image(image=[data], x=[0], y=[0], dh=[xr], dw=[yr],
                  palette="Spectral11")
tab1b = Panel(child=plot_snr_2d, title="SNR")

#saturation
data = out['2d']['saturation']
xr, yr = data.shape
plot_sat_2d = figure(tools=TOOLS,
                     x_range=[0, yr], y_range=[0, xr],
                     x_axis_label='Pixel', y_axis_label='Spatial',
                     title="Saturation",
                    plot_width=1000, plot_height=300)
plot_sat_2d.image(image=[data], x=[0], y=[0], dh=[xr], dw=[yr],
                  palette="Spectral11")
tab2b = Panel(child=plot_sat_2d, title="Saturation")

tabs2d = Tabs(tabs=[ tab1b, tab2b])
show(tabs2d)



# Color bar

In [ ]:
data = out['2d']['snr']
a = np.linspace(np.min(data),np.max(data),1000)

data = {' ': [' ']*1000 ,
        'fruit_count': a,
        '  ': a}
color = brewer["Spectral"][11][::-1]
hm = HeatMap(data, x=' ', y='  ', values='fruit_count',
             title='SNR', stat=None, color = color)
show(hm)


# EFFICIENCY PLOT (NOT DONE)

In [ ]:
#throughputs 

import get_pce as gp
instrument = 'nirspec'
mode = 'fixed_slit'
config = {
            'filter': 'f070lp',
            'aperture': 's1600a1',
            'disperser': 'g140m'}

nirsw1, nirsf1 = gp.get_pce(instrument, mode, config)

instrument = 'nirspec'
mode = 'fixed_slit'
config = {
            'filter': 'f100lp',
            'aperture': 's1600a1',
            'disperser': 'g140m'}

nirsw2, nirsf2 = gp.get_pce(instrument, mode, config)

instrument = 'nirspec'
mode = 'fixed_slit'
config = {
            'filter': 'f170lp',
            'aperture': 's1600a1',
            'disperser': 'g235m'}

nirsw3, nirsf3 = gp.get_pce(instrument, mode, config)

instrument = 'nirspec'
mode = 'fixed_slit'
config = {
            'filter': 'f290lp',
            'aperture': 's1600a1',
            'disperser': 'g395m'}

nirsw4, nirsf4 = gp.get_pce(instrument, mode, config)


instrument = 'miri'
mode = 'lrsslitless'
config = {
            'filter': None,
            'aperture': 'imager',
            'disperser': 'p750l'}

miriw, mirif = gp.get_pce(instrument, mode, config)


instrument = 'niriss'
mode = 'soss'
config = {
            'filter': None,
            'aperture': 'soss',
            'disperser': 'gr700xd_2'}

niriw1, nirif1 = gp.get_pce(instrument, mode, config)

instrument = 'niriss'
mode = 'soss'
config = {
            'filter': None,
            'aperture': 'soss',
            'disperser': 'gr700xd_1'}

niriw2, nirif2 = gp.get_pce(instrument, mode, config)


#NIRISS 



# PANDAS TIME TABLES

In [ ]:
timing = section5['timing']
timing_script = pd.DataFrame(timing.items()).to_html().encode()


In [ ]:
final = '<table class="table table-striped">'
timing_script =  '<table class="table table-striped"> \n'+ timing_script[36:len(timing_script)]
print timing_script

# Instrument tables 

In [ ]:
out = section5['out']
input_dict = {
    "Target J Mag": jmag , 
    "Saturation Level (electons)": sat_level, 
    "Instrument": out['input']['configuration']['instrument']['instrument'], 
    "Mode": out['input']['configuration']['instrument']['mode'], 
    "Aperture": out['input']['configuration']['instrument']['aperture'], 
    "Disperser": out['input']['configuration']['instrument']['disperser'], 
    "Subarray": out['input']['configuration']['detector']['subarray'], 
    "Readmode": out['input']['configuration']['detector']['readmode'], 
    "Filter": out['input']['configuration']['instrument']['filter']
}

In [ ]:
out['input']['configuration']['instrument']['mode']

# NOISE vs. JMAG

In [ ]:
cn = np.concatenate
g140 = pkl.load(open('nirspec140.pkl','r'))
g235 = pkl.load(open('nirspec235.pkl','r'))
g395 = pkl.load(open('nirspec395.pkl','r'))
prism = pkl.load(open('nirspecprism.pkl','r'))
f322 = pkl.load(open('nircam322.pkl','r'))
f444 = pkl.load(open('nircam444.pkl','r'))
niriss = pd.read_csv(open('niriss.txt'), index_col = False, header = None, delimiter=' ')

miri = pd.read_csv(open('miri.txt'), index_col = False, header = None, delimiter=' ')


In [ ]:
output_file("junk.html")
TOOLS = "pan,wheel_zoom,box_zoom,resize,reset,save"

jmagnoise = figure(tools=TOOLS,#responsive=True,
                     #x_axis_label='2MASS J',
                     #y_axis_label='Noise on Spectrum (ppm)', 
                       y_axis_type = 'log',
                     plot_width=400, plot_height=250)
'''
#1.4
jmagnoise.circle(g140['jmag'][1:len(g140['jmag'])], g140['1.4um'][1:len(g140['jmag'])]
                                                                 , alpha = .7, size = 10, color = 'black')
jmagnoise.line(g140['jmag'][1:len(g140['jmag'])], g140['1.4um'][1:len(g140['jmag'])]
               , line_width = 4, alpha = .7, color = 'black')
jmagnoise.triangle(niriss[0], niriss[1], alpha = .7, size = 10, color = 'black')
jmagnoise.line(niriss[0], niriss[1], line_width = 4, alpha = .7, color = 'black')
jmagnoise.diamond(prism['jmag'][3:len(prism['jmag'])], prism['1.4um'][3:len(prism['jmag'])], alpha = .7, size = 10, color = 'black')
jmagnoise.line(prism['jmag'][3:len(prism['jmag'])], prism['1.4um'][3:len(prism['jmag'])], line_width = 4, alpha = .7, color = 'black')

#2.5
jmagnoise.circle(g235['jmag'], g235['2.3um'],  alpha = .7,size = 10, color = 'blue')
jmagnoise.line(g235['jmag'], g235['2.3um'], line_width = 4, alpha = .7, color = 'blue')
jmagnoise.triangle(niriss[0], niriss[4],  alpha = .7,size = 10, color = 'blue')
jmagnoise.line(niriss[0], niriss[4], line_width = 4, alpha = .7, color = 'blue')
jmagnoise.diamond(prism['jmag'], prism['2.3um'],  alpha = .7, size = 10, color = 'blue')
jmagnoise.line(prism['jmag'], prism['2.3um'], line_width = 4, alpha = .7, color = 'blue')

#3.3
jmagnoise.square(f322['jmag'][2:len(prism['jmag'])], f322['3.3um'][2:len(prism['jmag'])], alpha = .7, size = 10, color = 'red')
jmagnoise.line(f322['jmag'][2:len(prism['jmag'])], f322['3.3um'][2:len(prism['jmag'])], line_width = 4, alpha = .7, color = 'red')
jmagnoise.diamond(prism['jmag'], prism['3.3um'], alpha = .7, size = 10, color = 'red')
jmagnoise.line(prism['jmag'], prism['3.3um'], line_width = 4, alpha = .7, color = 'red')
jmagnoise.circle(g395['jmag'], g395['3.3um'], alpha = .7,  size = 10, color = 'red')
jmagnoise.line(g395['jmag'], g395['3.3um'], line_width = 4, alpha = .7, color = 'red')
'''
#4.5
jmagnoise.square(f444['jmag'], f444['4.5um'], alpha = .7, size = 10, color = 'purple')
jmagnoise.line(f444['jmag'], f444['4.5um'], line_width = 4, alpha = .7, color = 'purple')
jmagnoise.diamond(prism['jmag'], prism['4.5um'], alpha = .7, size = 10, color = 'purple')
jmagnoise.line(prism['jmag'], prism['4.5um'], line_width = 4, alpha = .7, color = 'purple')
jmagnoise.circle(g395['jmag'], g395['4.5um'], alpha = .7,  size = 10, color = 'purple')
jmagnoise.line(g395['jmag'], g395['4.5um'], line_width = 4, alpha = .7, color = 'purple')
'''
#8um
jmagnoise.asterisk(miri[0], miri[3], alpha = .7, size = 10, color = 'green')
jmagnoise.line(miri[0], miri[3], line_width = 4, alpha = .7, color = 'green')

'''

show(jmagnoise)




In [3]:
import numpy as np
output_file('junk.html')
plot = figure()
x = range(0,1000)
y = range(0,1000)
source =ColumnDataSource(data=dict(x=x,y=y))
plot.circle('x','y',source=source)
show(plot)